# 科学技術計算 4 レポート


In [4]:
import math
import cmath
import numpy as np
import scipy
from typing import Tuple
from tqdm.auto import tqdm

rng = np.random.default_rng()

## 課題04-1

In [48]:
def lu_decomposition( A_org: np.ndarray ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """LU decomposition of A = PLU

    Args:
        A (np.ndarray): nxn matrix A

    Returns:
        (np.ndarray): nxn permutation matrix P
        (np.ndarray): nxn lower matrix L with 1s in diagonal
        (np.ndarray): nxn strictly upper matrix U
    """
    assert A_org.ndim == 2
    assert A_org.shape[0] == A_org.shape[1]

    #前進消去を行うコード
    n = A_org.shape[0]
    A = A_org.copy()
    print("A\n", A, sep="")
    L = np.eye(n)   #単位行列生成
    P = np.eye(n)   #置換行列

    for j in range(n - 1):
        #ここから資料にあった部分ピポッド選択月も前進消去の実装
          
        #print(f"{j}-th row", A[j])
        
        # 部分ピボット選択
        j_max = np.abs(A[j:, j]).argmax() + j   #行中の一番大きい値を抽出
        if j != j_max:
            # A の行交換
            A[[j, j_max]] = A[[j_max, j]]

            # L の行交換（前の列のみ）
            if j > 0:
                L[[j, j_max], :j] = L[[j_max, j], :j]

            # P の行交換 ← これが scipy と同じ P を作るカギ
            P[[j, j_max]] = P[[j_max, j]]
        
        #print("A\n", A, sep="")
        #print("P\n", P)
    
        # 前進消去
        remaining_rows = n - (j + 1)
        r_ij = A[j + 1:, j] / A[j, j]
        A[j + 1:, j + 1:] -= np.tile(A[j, j + 1:], (remaining_rows, 1)) * r_ij.reshape(remaining_rows, -1)
        A[j + 1:, j] = 0
        L[j + 1:, j] = r_ij
        #print("A\n", A, sep="")
        #$print("L\n", L, sep="")
        
    U = A
    
    #print("U\n", U, sep="")

    P = P.T
    """
    #確認１
    print("P =",P)
    print("L =", L)
    print("U =", U)

    print("LU\n", L @ U, sep="")
    print("A\n", A_org, sep="")
    #print("LU == A", np.allclose(L @ U, A_org))
    """
    return P, L, U



for i in tqdm(range(50)):
    n = rng.integers(low = 2, high = 10) #サイズ2-10
    A = rng.random(size=(n, n))

    P, L, U = lu_decomposition(A)
    Psp, Lsp, Usp = scipy.linalg.lu(A)
    """
    print("Psp = ", Psp)
    print("Lsp = ", Lsp)
    print("Usp = ", Usp)
    """
 
    assert np.allclose(L, Lsp), "values don't match"
    assert np.allclose(U, Usp), "values don't match"
    assert np.allclose(P, Psp), "values don't match"
    print("nyaaaaaaaaaaaaaaa")

  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.7950374  0.38337812 0.50243587 0.34737208 0.86473127 0.27443723]
 [0.86796766 0.53096551 0.35486841 0.58650263 0.51053886 0.08083723]
 [0.17374508 0.54067318 0.15909201 0.67340248 0.15827014 0.54965276]
 [0.2282263  0.78695611 0.41027162 0.19804296 0.7106573  0.63633592]
 [0.68812413 0.17565449 0.32018412 0.4282329  0.91652009 0.77832817]
 [0.81012313 0.9695028  0.66545775 0.75033552 0.64732479 0.74798414]]
nyaaaaaaaaaaaaaaa
A
[[0.09210733 0.53218516 0.51569464 0.12304937 0.60702736 0.94890348]
 [0.88761642 0.36121723 0.74095902 0.79744572 0.7706772  0.22615414]
 [0.47446196 0.62119282 0.24260609 0.37540841 0.12173274 0.88490197]
 [0.97881991 0.74535793 0.40310434 0.50434727 0.26488565 0.26472523]
 [0.18737392 0.92920164 0.43680647 0.92361255 0.44113808 0.10740278]
 [0.44796479 0.34712725 0.81953898 0.06752663 0.16694922 0.49523268]]
nyaaaaaaaaaaaaaaa
A
[[3.26096411e-01 9.81731989e-01 5.16079535e-01 1.67004140e-01
  6.73347965e-02 3.42785350e-01 4.41088228e-01 4.59447155e-01]
 [6

## 課題04-2


In [ ]:
def solve_lu( P: np.ndarray, L: np.ndarray, U: np.ndarray, b: np.ndarray,) -> np.ndarray:
    """Solve Ax = b with PLU, LU decomposition of A = PLU

    Args:
        P (np.ndarray): nxn permutation matrix P
        L (np.ndarray): nxn lower matrix L with 1s in diagonal
        U (np.ndarray): nxn strictly upper matrix U
        b (np.ndarray): n-d rhs vector b

    Returns:
        (np.ndarray): solution x of Ax = PLUx = b
    """
    assert b.ndim == 1
    n = b.shape[0]
    assert P.shape == (n, n)
    assert L.shape == (n, n)
    assert U.shape == (n, n)
    b = b.copy()

    # code here
    # solve Ly = P^T b
    a = P.T @ b
    # 前進代入
    y = np.zeros(n)
    for i in range(n):
        y[i] = a[i] - np.dot(L[i, :i], y[:i])
        # L[i,i] = 1（単位下三角行列）なので割り算不要

    # 後退代入
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    """
    x_true = solve(A, b)
    print("true solution x", x_true)
    print("x == x_true", np.allclose(x_true, x))
    print(x - x_true)
    """

    return x

for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))
    b = rng.random(n)

    P, L, U = lu_decomposition(A)
    x_with_lu = solve_lu(P, L, U, b) #こっちが自作

    x_numpy = np.linalg.solve(A, b) #こっちがライブラリにあるやつ

    assert np.allclose(x_numpy, x_with_lu ), "values don't match"

  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.01939665 0.69132636 0.59309245 0.60832725 0.78776907 0.68923275
  0.56498191 0.35508712 0.29882122]
 [0.857108   0.14787489 0.61244422 0.98597605 0.58231736 0.99290227
  0.14721314 0.08944359 0.80002837]
 [0.22754111 0.22436459 0.77826675 0.04078545 0.4350069  0.91382266
  0.85577363 0.74437576 0.50796782]
 [0.22175777 0.48716675 0.74082733 0.59773113 0.56889451 0.32126332
  0.86379143 0.11417666 0.73282681]
 [0.80814236 0.16050868 0.75467174 0.08397787 0.78421516 0.46375458
  0.75420393 0.69630889 0.06958219]
 [0.06990915 0.42614019 0.92977562 0.22374631 0.53247903 0.10858659
  0.50411592 0.0762273  0.94735772]
 [0.4379148  0.30359812 0.77719005 0.97037252 0.94827691 0.01803151
  0.07545027 0.66234347 0.57433977]
 [0.43065634 0.284585   0.69714311 0.08499533 0.93436668 0.73578926
  0.7293148  0.85373052 0.67331579]
 [0.95636875 0.35973893 0.44541869 0.59225468 0.7295329  0.57650265
  0.01572585 0.96864992 0.58531819]]
A
[[0.87130218 0.48151608 0.44120217 0.80041758 0.1288857  0.

## 課題04-4

In [54]:
def det_via_lu_decomposition(A: np.ndarray) -> float:
    """det(P) of permutation matrix P

    Args:
        P (np.ndarray): nxn permutation matrix of 0 and 1

    Returns:
        float: det(P) that is either 1 or -1
    """

    assert P.ndim == 2
    assert P.shape[0] == P.shape[1]

    for p in P.ravel()[np.nonzero(P.ravel())]:
        assert np.isclose(p, 1.0)
    
    n = P.shape[0]
    assert np.allclose(P.sum(axis=0), np.ones(n))
    assert np.allclose(P.sum(axis=1), np.ones(n))

    ## below three lines of code are helped with ChatGPT-4o because of its complexisity

    # Extract the permutation using np.nonzero
    permutation = np.nonzero(P)[1]

    # Calculate the number of inversions in the permutation
    inversions = sum(1 for i in range(len(permutation)) for j in range(i + 1, len(permutation)) if permutation[i] > permutation[j])

    # Return 1 if inversions are even, otherwise -1
    return 1 if inversions % 2 == 0 else -1

def det_lu( A: np.ndarray,) -> float:
    """compute det(A)

    Args:
        A (np.ndarray): nxn square matrix

    Returns:
        (float): det(A)
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    P, L, U = lu_decomposition(A)

    # code here

    return 0

for i in tqdm(range(50)):
    n = rng.integers(low=2, high=10)
    A = rng.random(size=(n, n))

    det_with_lu = det_lu(A)
    det_numpy = np.linalg.det(A)

    assert np.isclose( det_numpy, det_with_lu), "values don't match"


  0%|          | 0/50 [00:00<?, ?it/s]

A
[[0.41674671 0.8426097  0.42109403 0.68902854 0.24123511 0.2361761
  0.22231086]
 [0.97199377 0.38837705 0.69547953 0.78722581 0.65636527 0.03005566
  0.59780026]
 [0.55972166 0.28589722 0.85672977 0.69089454 0.69469295 0.61906559
  0.66317228]
 [0.53477106 0.51016916 0.87001017 0.6050124  0.28180315 0.14990043
  0.5702205 ]
 [0.66575722 0.97746342 0.84472195 0.88910394 0.02367749 0.27391558
  0.47130378]
 [0.41850783 0.55254975 0.65245957 0.61257837 0.11371252 0.79933283
  0.62525527]
 [0.95475704 0.37659695 0.45986213 0.42124521 0.09636108 0.06861354
  0.18289794]]


AssertionError: values don't match

## 課題04-7